<a href="https://colab.research.google.com/github/cyberseihis/bnnprint/blob/main/notebooks/qkerastnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install keras-tuner QKeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyparser: filename=pyparser-1.0-py3-none-any.whl size=4941 sha256=03eda82a9c2d1f68875d22283bf7284879789c6ce6938867333b4e6ebbbe9519
  Stored in directory: /root/.cache/pip/wheels/3f/a4/1a/fdbe78211760c69a9c7635d224d3683c8ed2519e01f53a81e0
  Created wheel for parse: filename=parse-1.6.5-py3-none-any.whl size=18174 sha256=a508d9c3bf3f33912886f7f06d47bb3a0fd8ccfaef7302031ceb664d7c93d423
  Stored in directory: /root/.cache/pip/wheels/37/ef/ff/5004e21f680c6489785b14199d1716e5db6eb05ca497068b1f
Successfully built pyparser parse


In [ ]:
def get_csv_filenames():
    filenames = os.listdir()
    csv_filenames = [os.path.splitext(filename)[0] for filename in filenames if filename.endswith(".csv")]
    return csv_filenames

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner import HyperParameters
import keras_tuner as kt
from tensorflow.keras.utils import to_categorical

import numpy as np
import pandas as pd
import os

# data and metric imports
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics

from qkeras import *
import qkeras.utils as qutils




def build_model(input_shape, output_shape, hp):
    model = keras.Sequential()
    model.add(QActivation("quantized_bits(4, keep_negative=0)"))
    model.add(QDense(units=40,kernel_quantizer="ternary", use_bias=False, input_shape=(input_shape,)))
    #model.add(layers.BatchNormalization(scale=False))
    #model.add(layers.Activation('relu'))
    model.add(QActivation("binary"))
    #model.add(layers.Dense(units=output_shape))
    #model.add(layers.Dropout(rate=hp.Float("dropout", min_value=0, max_value=0.4)))
    model.add(QDense(units=output_shape, kernel_quantizer="ternary(alpha=1)", use_bias=False))
    model.add(layers.Activation('softmax'))

    model.compile(
        #maybe I could add more hyperparameters
        optimizer=keras.optimizers.Adam(
            hp.Float("learning_rate", min_value=0.001, max_value=0.02)),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model


def train_csv(fname,tri=20,ep=50):
    df = pd.read_csv(fname+'.csv', header=None)
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    X = pd.DataFrame(X.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))))
    X_train, X_test, y_train, y_test = \
        sklearn.model_selection.train_test_split(X, y, test_size=0.3, random_state=42)
    (_, input_shape) = X.shape
    output_shape = y.nunique()
    y_train = to_categorical(y_train, num_classes=output_shape)
    y_test = to_categorical(y_test, num_classes=output_shape)

    def my_builder(hp):
        return build_model(input_shape, output_shape, hp)

    tuna = kt.BayesianOptimization(
        hypermodel=my_builder,
        objective="val_accuracy",
        max_trials=tri,
        project_name="proj"+fname
    )
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    tuna.search(X_train, y_train, epochs=ep, validation_data=(X_test, y_test), callbacks=[early_stop])
    print(fname+"summary")
    #tuna.results_summary()
    model = tuna.get_best_models(num_models=1)[0]
    x_hack = tf.random.normal(shape=(1,input_shape),dtype=tf.float32)
    output=model.predict(x_hack)
    qutils.model_save_quantized_weights(model,'weightnn'+fname+'.h5')
    model.save(fname+"tnn.h5")
    model.evaluate(X_test, y_test)

In [ ]:
/os.chdir('drive/MyDrive/colab notebooks/dataset')

In [ ]:
train_csv("less_winered")

Trial 20 Complete [00h 00m 09s]
val_accuracy: 0.5666666626930237

Best val_accuracy So Far: 0.5708333253860474
Total elapsed time: 00h 02m 17s
less_wineredsummary
1/1 [==============================] - 1s 717ms/step
... quantizing model
15/15 [==============================] - 0s 4ms/step - loss: 1.2332 - accuracy: 0.5708


In [ ]:
for fname in get_csv_filenames():
  train_csv(fname)

less_pendigitssummary


1/1 [==============================] - 1s 681ms/step
... quantizing model
71/71 [==============================] - 1s 3ms/step - loss: 0.5281 - accuracy: 0.8395
less_cardiosummary


1/1 [==============================] - 1s 683ms/step
... quantizing model
20/20 [==============================] - 0s 4ms/step - loss: 0.4019 - accuracy: 0.8824
less_Harsummary


1/1 [==============================] - 1s 682ms/step
... quantizing model
28/28 [==============================] - 0s 4ms/step - loss: 1.1668 - accuracy: 0.5333
less_wineredsummary


1/1 [==============================] - 1s 663ms/step
... quantizing model
15/15 [==============================] - 0s 3ms/step - loss: 1.2332 - accuracy: 0.5708
less_gasIdsummary


1/1 [==============================] - 1s 669ms/step
... quantizing model
131/131 [==============================] - 1s 3ms/step - loss: 0.5700 - accuracy: 0.8212
less_winequality_whitesummary


1/1 [==============================] - 1s 664ms/step
... quantizing model
46/46 [==============================] - 0s 3ms/step - loss: 1.6193 - accuracy: 0.5048


In [ ]:
df = pd.read_csv('winequality_white.csv', header=None)
Xx = df.iloc[:, :-1]
y = df.iloc[:, -1]
X = pd.DataFrame(Xx.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))))

In [ ]:
%pip install h5py


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from keras.models import load_model

# Load the model from the HDF5 file
model = load_model('winequality_whitetnn.h5', custom_objects={'QActivation': QActivation,'QDense':QDense})

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [ ]:
#x_hack = tf.random.normal(shape=(1,16),dtype=tf.float32)
#x_hack = tf.abs(x_hack)
#output=model.predict(x_hack)
#inpux = np.expand_dims(X.iloc[0], axis=0)
inpux = X.iloc[0:1]
lo=get_layer_outputs(model,inpux)
yoyo = lo[2]
gol = lo[4]
lo

[array([[0.3125, 0.1875, 0.1875, 0.3125, 0.125 , 0.125 , 0.375 , 0.25  ,
         0.25  , 0.25  , 0.125 ]], dtype=float32),
 array([[-0.15625 ,  1.625   , -0.03125 , -0.25    ,  0.09375 , -0.1875  ,
          0.59375 , -0.25    , -0.09375 ,  0.      ,  0.28125 ,  0.15625 ,
          0.1875  ,  0.15625 ,  0.03125 , -0.21875 ,  1.9375  , -0.1875  ,
          0.234375, -0.21875 ,  0.      ,  0.15625 ,  0.125   ,  1.625   ,
          0.0625  , -0.0625  , -0.3125  ,  0.4375  , -0.0625  , -0.4375  ,
          0.4375  ,  0.      ,  0.53125 ,  0.28125 , -0.09375 ,  0.21875 ,
         -0.25    , -0.125   ,  0.125   , -0.078125]], dtype=float32),
 array([[-1.,  1., -1., -1.,  1., -1.,  1., -1., -1.,  1.,  1.,  1.,  1.,
          1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1.,
         -1.,  1., -1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1.,  1.,
         -1.]], dtype=float32),
 array([[-6., -3.,  0.,  2., -3., -4., -5.]], dtype=float32),
 array([[2.9106444e-04, 5.8461856e-03, 

In [ ]:
xrisos = np.argmax(gol,axis=1)
qinp = lo[0]

In [ ]:
import h5py

filename = "weightnnwinequality_white.h5"

h5 = h5py.File(filename,'r')

h5.visit(print)
#print(h5['q_dense/q_dense/kernel:0'][:,:])
#print(h5['q_batch_normalization/q_batch_normalization/beta:0'][:])
sw0 = h5['q_dense/q_dense/kernel:0'][:,:]
sw1 = h5['q_dense_1/q_dense_1/kernel:0'][:,:]

h5.close()

activation
q_activation
q_activation_1
q_dense
q_dense/q_dense
q_dense/q_dense/kernel:0
q_dense_1
q_dense_1/q_dense_1
q_dense_1/q_dense_1/kernel:0
top_level_model_weights


In [ ]:
matz=np.sign(sw0)
abz=np.abs(matz)
sinz=np.maximum(matz,0)
binst(abz.flatten())[::-1]

'10011001110000000100111010000101000100011001101110011010111111111100011111111110011001101001111010110011101101001011101001001110011000111011011011110110101101001111101101010110110110010101001010110101001110011011110011011011100001110110110101110111111100111011101110010000001111011111011101101011111011011100011111011001101000001101010101100111000001000001000111101010100110101001010000010110000111000110100101011110101011111110110100000011'

In [ ]:
cnz=np.count_nonzero(sw0, axis=0)
''.join([f"0{i}" for i in cnz])

'06070806070407070707040806050804090607070707080808050305010906050503060906060307'

In [ ]:
hay=binst(yoyo[0])
jay = '0100101001111110101011111001001111010010'[::-1]
jay

'0100101111001001111101010111111001010010'

In [ ]:
def dbytes(mat):
  kay=mat.flatten().astype(np.uint8).tobytes()
  return binascii.hexlify(kay).decode()

In [ ]:
from scipy.sparse import csr_matrix
spa=csr_matrix(sw1.T)
#spa.data,spa.indices,spa.indptr
svals=binst(np.maximum(spa.data,0))
scols=dbytes(spa.indices[::-1])
srows=dbytes(spa.indptr[::-1])
svals,scols,srows

('0100000010110010011000100000110000101111111100111011000110111010101011011001010',
 '2625232221201e1b1817141211100d0c0605040201211f17100d0b211c1615100c0825211f1817100f0125242119181412092625221f1e1c1817161413100b0a07261c171412100f0b0a0907060401',
 '4f3a342d251d0e00')

In [ ]:
lo[3]

array([[-6., -3.,  0.,  2., -3., -4., -5.]], dtype=float32)

In [ ]:
sw1.T

array([[ 0., -1.,  0.,  0.,  1.,  0., -1.,  1.,  0., -1., -1.,  1.,  0.,
         0.,  0.,  1., -1.,  0.,  1.,  0.,  1.,  0.,  0., -1.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -1.,  1.,  0.,
         0.,  0.,  0., -1.,  0.,  0.,  1.,  1.,  0.,  1., -1.,  1.,  0.,
         0.,  0.,  1.,  0., -1., -1.,  0.,  0., -1.,  0.,  0.,  1.,  1.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1., -1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  1.,  1.,  0.,
         0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0., -1.,  0.,  0.,  0.,  1.,  0.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0., 

In [ ]:
nn=np.count_nonzero(sw1, axis=0)
oh=(lo[3]+nn)/2
lo[3],oh

(array([[-6., -3.,  0.,  2., -3., -4., -5.]], dtype=float32),
 array([[4., 6., 4., 5., 2., 1., 8.]]))

In [ ]:
np.max(np.abs(sw0.T),axis=1)

array([0.25, 1.  , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.25, 0.5 ,
       0.25, 0.5 , 0.5 , 0.5 , 0.5 , 1.  , 0.5 , 0.25, 0.5 , 0.5 , 0.5 ,
       0.5 , 1.  , 0.25, 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 ,
       0.5 , 0.5 , 0.25, 0.5 , 0.5 , 0.5 , 0.25], dtype=float32)

In [ ]:
def mysign(a):
  signs = np.sign(a)

  # Replace the sign of the zeroes with 1 (positive) or -1 (negative)
  signs_no_zero = np.where(signs == 0, 1, signs)
  return signs_no_zero

In [ ]:
def grup(x):
  x1 = x @ sw0
  x2 = mysign(x1)
  x3 = x2 @ sw1
  return (x1,x2,x3)
def drut(x):
  x1 = x @ np.sign(sw0)
  x2 = mysign(x1)
  x3 = x2 @ sw1
  am = np.argmax(x3,axis=1)
  return am

In [ ]:
xrisos[0:20]

array([3, 3, 2, 3, 3])

In [ ]:
drut(qinp)[0:20]

array([3, 3, 2, 3, 3])

In [ ]:
np.where(np.not_equal(yoyo, grup(qinp)[1]))

(array([], dtype=int64), array([], dtype=int64))

In [ ]:
import binascii
def binst(yy):
  yozo = np.maximum(yy,0)
  asbin=yozo.astype(np.uint8).tobytes()
  bs=binascii.hexlify(asbin).decode()[1::2]
  return bs[::-1]

[print(binst(yy)) for yy in yoyo]

0100101111001001111101010111111001010010


[None]

In [ ]:
y[0:10]

0    3
1    3
2    3
3    3
4    3
5    3
6    3
7    3
8    3
9    3
Name: 11, dtype: int64

In [ ]:
glo = lo[2][175]
gor = grup(qinp[175])[1]
glo, gor, np.where(np.not_equal(glo,gor))

(array([-1., -1., -1., -1., -1., -1., -1., -1.,  1.,  1.,  1., -1.,  1.,
        -1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
         1.,  1., -1., -1., -1.,  1.,  1., -1.,  1., -1., -1., -1., -1.,
        -1.], dtype=float32),
 array([-1., -1., -1., -1., -1., -1., -1., -1.,  1.,  0.,  0., -1.,  1.,
        -1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
         1.,  1., -1., -1., -1.,  1.,  0., -1.,  1., -1., -1., -1., -1.,
        -1.], dtype=float32),
 (array([ 9, 10, 32]),))

In [ ]:
#dut = yoyo @ (np.sign(swax))
#dd , gg = np.argmax(dut,axis=1),np.argmax(gol,axis=1)
hm=np.where(np.not_equal(drut(qinp),y[0:199]))
ex=np.where(np.not_equal(xrisos,y[0:199]))
#(dd[hm],gg[hm],hm)
hm,ex,np.where(np.not_equal(hm,ex))

((array([ 13,  27,  29,  49,  74,  82,  86,  93, 114, 148, 152, 154, 160,
         161, 162, 166, 180]),),
 (array([ 13,  27,  29,  49,  74,  82,  86,  93, 114, 148, 152, 154, 160,
         161, 162, 166, 180]),),
 (array([], dtype=int64), array([], dtype=int64)))

In [ ]:
h5.keys()

ValueError: ignored

In [ ]:
h5['q_dense_1/q_dense_1/kernel:0'][:,:]

ValueError: ignored

In [ ]:
list(QBatchNormalization().quantizers)

In [ ]:
import keras.backend as K

def get_layer_outputs(model, x):
    """
    Returns the output of every layer in a Keras model evaluated at the input x.
    
    Args:
    model: A Keras model object.
    x: The input to the model as a Numpy array or a list of Numpy arrays.
    
    Returns:
    A list of Numpy arrays, where the i-th element is the output of the i-th layer
    in the model evaluated at the input x.
    """
    # Create a function that computes the output of every layer given the input
    layer_outputs = [layer.output for layer in model.layers]
    layer_eval_fn = K.function(inputs=[model.input], outputs=layer_outputs)
    
    # Evaluate the function at the input
    output_values = layer_eval_fn([x])
    
    return output_values


In [ ]:
from keras.models import load_model
def get_results(fname):
    df = pd.read_csv(fname+".csv", header=None)
    X = df.iloc[:, :-1]
    
    X = pd.DataFrame(X.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))))
    
    model = load_model(fname+"tnn.h5", custom_objects={'QActivation': QActivation,'QDense':QDense})
   
    output=model.predict(X)
    clases = np.argmax(output, axis=1)
    return clases

In [ ]:
get_results("pendigits")[0:14]

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


235/235 [==============================] - 4s 2ms/step


array([8, 2, 1, 4, 1, 6, 4, 0, 5, 0, 9, 8, 5, 5])

In [ ]:
def write_strings_to_file(strings, file_path):
    with open(file_path, 'w') as file:
        for string in strings:
            file.write(str(string) + '\n')

In [ ]:
for fname in get_csv_filenames():
  gres = get_results(fname).tolist()
  write_strings_to_file(gres,'results_'+fname+'.results')

/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


235/235 [==============================] - 1s 3ms/step


/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


67/67 [==============================] - 0s 2ms/step


/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


...

50/50 [==============================] - 0s 2ms/step


/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


435/435 [==============================] - 1s 2ms/step


/usr/local/lib/python3.9/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


93/93 [==============================] - 0s 2ms/step


In [ ]:
inpux = X.iloc[0:1]
#inpux = np.array([[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]])
lo=get_layer_outputs(model,inpux)
yoyo = lo[2]
gol = lo[4]
lo